In [1]:
# To see views.parquet
import pandas as pd
df = pd.read_parquet("../data/processed/views.parquet")
print(df.shape)                # (nb_rtis, 5)
print(df.columns.tolist())     # ['rti_number','gti_number','title_text','chap_titles_text','meta_text']
print(df.head(3).to_string())


(10, 8)
['rti_number', 'gti_number', 'title_text', 'chap_titles_text', 'meta_text', 'lang_title', 'lang_chapters', 'lang_rti']
  rti_number gti_number               title_text     chap_titles_text meta_text lang_title lang_chapters lang_rti
0     RTI001     GTI001   Hydraulic Pump Failure  Failure Description    8 | QA         en            en       en
1     RTI002     GTI001  Hydraulic Pressure Drop             Symptoms    5 | QC         en            en       en
2     RTI003     GTI002   Avionics Cooling Issue                         2 | QA         en          None       en


In [9]:
import sys, os
# depuis research/, on remonte d’un cran et on ajoute src
sys.path.append(os.path.abspath(os.path.join("..", "src")))

from rti_sim.embedding.textcraft import build_embedding_text
import pandas as pd

df = pd.read_parquet("../data/processed/views.parquet")
s = build_embedding_text(df)
print(s.head(2).tolist())


['passage: title: Hydraulic Pump Failure Hydraulic Pump Failure \nchapters: Failure Description \nmeta: 8 | QA', 'passage: title: Hydraulic Pressure Drop Hydraulic Pressure Drop \nchapters: Symptoms \nmeta: 5 | QC']


In [12]:
from rti_sim.embedding.encoder import TextEncoder
enc = TextEncoder("intfloat/multilingual-e5-base", device="auto", normalize=True, batch_size=64)
vecs = enc.encode(s.tolist())
print(vecs.shape)  # (N, d)


c:\Users\Mehdi\AppData\Local\Programs\Python\Python38\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\Mehdi\AppData\Local\Programs\Python\Python38\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] La procédure spécifiée est introuvable'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
c:\Users\Mehdi\AppData\Local\Programs\Python\Python38\lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(10, 768)


In [13]:
vecs

array([[ 0.01354055,  0.01274305, -0.02883781, ..., -0.04925426,
        -0.03922504,  0.03981287],
       [ 0.00403146,  0.03340651, -0.02382699, ..., -0.04250335,
        -0.04505576,  0.06114832],
       [-0.00563614,  0.02758808, -0.0073838 , ..., -0.02724185,
        -0.05500587,  0.03793624],
       ...,
       [-0.01012027,  0.04143563, -0.02608121, ..., -0.03080353,
        -0.02408315,  0.05187466],
       [-0.00991028,  0.0251252 , -0.01182845, ..., -0.03916492,
        -0.02931739,  0.03451868],
       [ 0.0085158 ,  0.03769193, -0.01580486, ..., -0.04864909,
        -0.06883626,  0.03085272]], dtype=float32)

In [15]:
import numpy as np, pandas as pd

E = np.load("../artifacts/catalog_embeddings.npy")          # (10, 768)
M = pd.read_parquet("../artifacts/catalog_meta.parquet")    # 10 rows
assert E.shape[0] == len(M)                              # row alignment check

# cosine sims (normalized embeddings → dot = cosine)
S = E @ E.T
np.fill_diagonal(S, 0.0)
top1 = S.argmax(axis=1)
print(list(zip(M["rti_number"], M["gti_number"], M["rti_number"].iloc[top1], M["gti_number"].iloc[top1])))


[('RTI001', 'GTI001', 'RTI002', 'GTI001'), ('RTI002', 'GTI001', 'RTI001', 'GTI001'), ('RTI003', 'GTI002', 'RTI007', 'GTI004'), ('RTI004', 'GTI002', 'RTI003', 'GTI002'), ('RTI005', 'GTI003', 'RTI003', 'GTI002'), ('RTI006', 'GTI003', 'RTI005', 'GTI003'), ('RTI007', 'GTI004', 'RTI003', 'GTI002'), ('RTI008', 'GTI004', 'RTI007', 'GTI004'), ('RTI009', 'GTI005', 'RTI003', 'GTI002'), ('RTI010', 'GTI005', 'RTI005', 'GTI003')]


In [18]:
import pandas as pd
df = pd.read_parquet("../out/neighbors.parquet")
print(df.to_string())
# columns: query_rti, candidate_rti, candidate_gti, score, rank


   query_rti candidate_rti candidate_gti     score  rank
0     RTI001        RTI001        GTI001  1.000000     1
1     RTI001        RTI002        GTI001  0.944955     2
2     RTI001        RTI006        GTI003  0.914715     3
3     RTI001        RTI007        GTI004  0.909822     4
4     RTI001        RTI005        GTI003  0.906522     5
5     RTI002        RTI002        GTI001  1.000000     1
6     RTI002        RTI001        GTI001  0.944955     2
7     RTI002        RTI006        GTI003  0.920350     3
8     RTI002        RTI007        GTI004  0.917593     4
9     RTI002        RTI003        GTI002  0.910416     5
10    RTI003        RTI003        GTI002  1.000000     1
11    RTI003        RTI007        GTI004  0.936470     2
12    RTI003        RTI005        GTI003  0.933283     3
13    RTI003        RTI008        GTI004  0.929737     4
14    RTI003        RTI004        GTI002  0.925774     5
15    RTI004        RTI004        GTI002  1.000000     1
16    RTI004        RTI003     

In [1]:
import pandas as pd, numpy as np, json
a = pd.read_parquet("../artifacts/cluster_assignments.parquet")
print(a.groupby("cluster_id").size())
print(a.head().to_string())

C = np.load("../artifacts/cluster_centroids.npy")
print("centroids:", C.shape)

print(json.load(open("../artifacts/cluster_meta.json")))
b = pd.read_parquet("../out/cluster_assignments_new.parquet")
print(b.head().to_string())


cluster_id
0    1
1    6
2    2
3    1
dtype: int64
  rti_number gti_number  cluster_id  sim_to_centroid
0     RTI001     GTI001           2         0.986143
1     RTI002     GTI001           2         0.986143
2     RTI003     GTI002           1         0.971746
3     RTI004     GTI002           0         1.000000
4     RTI005     GTI003           1         0.972507
centroids: (4, 768)
{'algorithm': 'kmeans', 'k': 4, 'random_state': 42, 'max_iter': 300, 'silhouette_euclid': 0.07928554713726044, 'davies_bouldin': 1.0461160335680513, 'n': 10, 'd': 768}
  rti_number  assigned_cluster     score  rank
0     RTI001                 2  0.986143     1
1     RTI002                 2  0.986143     1
2     RTI003                 1  0.971746     1
3     RTI004                 0  1.000000     1
4     RTI005                 1  0.972507     1


In [2]:
import pandas as pd, json
s = pd.read_parquet("../artifacts/cluster_summary.parquet")
print(s.columns.tolist())
print(s[["cluster_id","size","major_gti","purity","avg_sim"]].head().to_string())

d = pd.read_parquet("../artifacts/cluster_gti_distribution.parquet")
assert {"cluster_id","gti_number","count","pct"} <= set(d.columns)

['cluster_id', 'size', 'avg_sim', 'major_gti', 'purity', 'top_terms', 'exemplars']
   cluster_id  size major_gti    purity   avg_sim
0           0     1    GTI002  1.000000  1.000000
1           1     6    GTI003  0.333333  0.966582
2           2     2    GTI001  1.000000  0.986143
3           3     1    GTI005  1.000000  1.000000
